# Suspicious Scripting in a WMI Consumer
Detects suspicious scripting in WMI Event Consumers

## Rule Content
```
- title: Suspicious Scripting in a WMI Consumer
  id: fe21810c-2a8c-478f-8dd3-5a287fb2a0e0
  status: experimental
  description: Detects suspicious scripting in WMI Event Consumers
  author: Florian Roth
  references:
  - https://in.security/an-intro-into-abusing-and-identifying-wmi-event-subscriptions-for-persistence/
  - https://github.com/Neo23x0/signature-base/blob/master/yara/gen_susp_lnk_files.yar#L19
  date: 2019/04/15
  tags:
  - attack.t1086
  - attack.execution
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 20
      Destination:
      - '*new-object system.net.webclient).downloadstring(*'
      - '*new-object system.net.webclient).downloadfile(*'
      - '*new-object net.webclient).downloadstring(*'
      - '*new-object net.webclient).downloadfile(*'
      - '* iex(*'
      - '*WScript.shell*'
      - '* -nop *'
      - '* -noprofile *'
      - '* -decode *'
      - '* -enc *'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Administrative scripts
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"20" AND wmi_consumer_destination.keyword:(*new\-object\ system.net.webclient\).downloadstring\(* OR *new\-object\ system.net.webclient\).downloadfile\(* OR *new\-object\ net.webclient\).downloadstring\(* OR *new\-object\ net.webclient\).downloadfile\(* OR *\ iex\(* OR *WScript.shell* OR *\ \-nop\ * OR *\ \-noprofile\ * OR *\ \-decode\ * OR *\ \-enc\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()